In [ ]:
import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import KFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
import warnings
warnings.filterwarnings('ignore', message='X does not have valid feature names')

In [ ]:
year = input("Insert a year or avg")

Insert a year or avgavg


In [ ]:
avtopati = pd.read_csv(f"avtopati_{year}.csv")
avtopati.head()

,RB,R_NUMBER,S_NUMBER,S_NAME,Datum,LENGTH SECTION,Ave_Rain,Max_Snow,Ave_Temp,Max_Temp,...,PCI,KON.NAK.,SSD H,SSD V,V sign,H sign,K.Int. P.T,K.Bridges,PGDS_AVG,Wi_AVG
0,1,A1,1,Drzhavna granica RS/MK (Tabanovce) - Rechica (...,2013,5664,602,10.85,12.69,30.98,...,92,0.53,0.88,0.18,12,2.0,0.18,1.06,7136.2,2.2069
1,2,A1,2,Rechica (avtopat) - D. Konjare 1 (avtopat),2013,3489,596,10.09,12.82,31.16,...,91,0.29,1.15,0.00,12,2.0,0.57,0.86,7136.2,1.1464
2,3,A1,3,D. Konjare 1 (avtopat) - Kumanovo 1 (avtopat),2013,2775,593,9.67,12.89,31.25,...,79,0.72,0.72,0.36,12,2.0,0.36,1.44,7136.2,9.3694
3,4,A1,4,Kumanovo 1 (avtopat) - Miladinovci 1 (avtopat),2016,17855,598,10.27,12.79,31.11,...,76,0.62,0.73,0.11,12,2.0,0.17,0.78,8112.4,2.4643
4,5,A1,5,Miladinovci 1 (avtopat) - Petrovec 1 (avtopat),2017,5827,569,6.85,13.38,31.93,...,71,0.17,0.51,0.00,21,2.0,0.51,0.86,7893.2,3.3465


In [ ]:
avtopati.columns

Index(['RB', 'R_NUMBER', 'S_NUMBER', 'S_NAME', 'Datum', 'LENGTH SECTION',
       'Ave_Rain', 'Max_Snow', 'Ave_Temp', 'Max_Temp', 'Min_Temp',
       'Аve_Height', 'LIMIT', 'Kr.Kar.', 'Ave_Rad', 'Lat_Force', 'Ave_Inc',
       'SKIDRES', 'RDB_RUT ', 'IRI', 'PCI', 'KON.NAK.', 'SSD H', 'SSD V',
       'V sign', 'H sign', 'K.Int. P.T', 'K.Bridges', 'PGDS_AVG', 'Wi_AVG'],
      dtype='object')

In [ ]:
feats = pd.read_csv(f"feature_importance_{year}.csv")
feats.head()

,feature,importance_mean,importance_std,importance_mean_clipped,percentage
0,LENGTH SECTION,0.125,0.012,0.125,17.000
1,PCI,0.053,0.005,0.053,7.186
2,LIMIT,0.049,0.008,0.049,6.624
3,K.Int. P.T,0.046,0.005,0.046,6.189
4,K.Bridges,0.036,0.004,0.036,4.943


In [ ]:
def get_features_to_drop(f, threshold=80.0, feature_col="feature", importance_col="percentage"):

    df_sorted = f.sort_values(
        by=importance_col,
        ascending=False
    ).reset_index(drop=True)

    df_sorted["cumulative_importance"] = df_sorted[importance_col].cumsum()

    kept_features = df_sorted.loc[
        df_sorted["cumulative_importance"] <= threshold,
        feature_col
    ].tolist()

    if df_sorted.loc[len(kept_features), "cumulative_importance"] > threshold:
        kept_features.append(
            df_sorted.loc[len(kept_features), feature_col]
        )

    dropped_features = list(
        set(f[feature_col]) - set(kept_features)
    )

    return kept_features, dropped_features

In [ ]:
TARGET_COL = f"Wi_{year.upper()}"

thresholds = [70, 80, 90]

feature_sets = {}

for t in thresholds:
    kept, dropped = get_features_to_drop(
        feats,
        threshold=t,
        feature_col="feature",
        importance_col="percentage"
    )
    feature_sets[t] = kept

    print(f"\nTop {t}% features ({len(kept)}):")
    print(kept)



Top 70% features (12):
['LENGTH SECTION', 'PCI', 'LIMIT', 'K.Int. P.T', 'K.Bridges', 'PGDS_AVG', 'Max_Snow', 'Аve_Height', 'Ave_Temp', 'Max_Temp', 'Ave_Inc', 'Ave_Rain']

Top 80% features (15):
['LENGTH SECTION', 'PCI', 'LIMIT', 'K.Int. P.T', 'K.Bridges', 'PGDS_AVG', 'Max_Snow', 'Аve_Height', 'Ave_Temp', 'Max_Temp', 'Ave_Inc', 'Ave_Rain', 'Min_Temp', 'IRI', 'SSD H']

Top 90% features (19):
['LENGTH SECTION', 'PCI', 'LIMIT', 'K.Int. P.T', 'K.Bridges', 'PGDS_AVG', 'Max_Snow', 'Аve_Height', 'Ave_Temp', 'Max_Temp', 'Ave_Inc', 'Ave_Rain', 'Min_Temp', 'IRI', 'SSD H', 'RB', 'S_NUMBER', 'RDB_RUT ', 'KON.NAK.']


In [ ]:
def prepare_xy_kfold(df, features, target=TARGET_COL, n_splits=5, random_state=42):
    df = df.dropna()
    X = df[features].values
    y = df[target].values

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        yield X_train, X_test, y_train, y_test

In [ ]:
models = {
    "Linear": LinearRegression(),
    "MLP": MLPRegressor(
        hidden_layer_sizes=(100, 50),
        activation='relu',
        solver='adam',
        alpha=0.0001,
        batch_size='auto',
        learning_rate='adaptive',
        max_iter=500,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1
    ),
        "XGBoost": XGBRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=6,
        random_state=42,
        n_jobs=-1
    ),
    "LightGBM": LGBMRegressor(
        n_estimators=200,
        learning_rate=0.05,
        random_state=42,
        n_jobs=-1,
        verbose=-1,
    ),
    "RandomForest": RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ),
    "GradientBoost": GradientBoostingRegressor(
        random_state=42
    )
}

In [ ]:
results = []
all_predictions = {}

for t, features in feature_sets.items():
    fold_metrics = {name: [] for name in models.keys()}
    fold_predictions = {name: {'actual': [], 'predicted': []} for name in models.keys()}

    for X_train, X_test, y_train, y_test in prepare_xy_kfold(avtopati, features):
        for name, model in models.items():
            model.fit(X_train, y_train)
            preds = model.predict(X_test)

            fold_predictions[name]['actual'].extend(y_test)
            fold_predictions[name]['predicted'].extend(preds)

            fold_metrics[name].append({
                "RMSE": np.sqrt(mean_squared_error(y_test, preds)),
                "MAE": mean_absolute_error(y_test, preds),
                "R2": r2_score(y_test, preds)
            })

    all_predictions[t] = fold_predictions

    for name, metrics in fold_metrics.items():
        avg_rmse = np.mean([m["RMSE"] for m in metrics])
        avg_mae  = np.mean([m["MAE"] for m in metrics])
        avg_r2   = np.mean([m["R2"] for m in metrics])

        results.append({
            "features_%": t,
            "model": name,
            "RMSE": avg_rmse,
            "MAE": avg_mae,
            "R2": avg_r2
        })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="RMSE").reset_index(drop=True)
results_df

,features_%,model,RMSE,MAE,R2
0,70,GradientBoost,6.967334,4.303427,0.304733
1,90,GradientBoost,7.069503,4.219978,0.332608
2,80,GradientBoost,7.122780,4.349550,0.307069
3,70,RandomForest,8.183691,4.756668,0.142856
4,80,RandomForest,8.300959,4.849510,0.100170
5,90,RandomForest,8.385743,4.798261,0.109821
6,90,XGBoost,8.417068,4.714006,0.147073
7,70,XGBoost,8.856656,4.962769,-0.003457
8,80,XGBoost,9.010421,5.024169,-0.002493
9,70,LightGBM,9.168861,5.265628,-0.259722


Најдобар од пробаните модели е GradientBoost кога се земаат најважните 70% податоци.

In [ ]:
gb_70_predictions = pd.DataFrame({
    'Actual': all_predictions[70]['GradientBoost']['actual'],
    'Predicted': all_predictions[70]['GradientBoost']['predicted']
})

print(f"Дефинираните тежински индекси користејќи ги највлијателните податоци: ")
print(gb_70_predictions.head(20))

Дефинираните тежински индекси користејќи ги највлијателните податоци: 
      Actual  Predicted
0     4.0900   0.626402
1     1.0995  -0.250916
2     5.6872   0.831847
3     8.0062   5.899837
4     5.7308   7.837641
5     9.9786   6.319733
6     0.6246   0.634667
7     5.8875   0.315535
8     2.4853   1.625800
9    12.9703   8.564389
10   11.3260  11.743719
11  105.6783  32.263406
12    5.4302   6.480480
13   13.8126   6.890345
14    5.8997  40.836328
15   20.0000  19.512555
16    6.0976   9.961973
17    9.4614   4.495736
18    6.8869   9.739271
19    4.1792   5.763014


In [ ]:
best_row = results_df.iloc[0]
best_threshold = best_row['features_%']
best_model_name = best_row['model']

print(f"Best model: {best_model_name} with {best_threshold}% features")
print(f"RMSE: {best_row['RMSE']:.4f}, R2: {best_row['R2']:.4f}")

best_features = feature_sets[best_threshold]

df_clean = avtopati.dropna()
X_full = df_clean[best_features].values
y_full = df_clean[TARGET_COL].values

scaler = StandardScaler()
X_full_scaled = scaler.fit_transform(X_full)

if best_model_name == "LightGBM":
    final_model = LGBMRegressor(
        n_estimators=200,
        learning_rate=0.05,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
elif best_model_name == "XGBoost":
    final_model = XGBRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=6,
        random_state=42,
        n_jobs=-1
    )
elif best_model_name == "GradientBoost":
    final_model = GradientBoostingRegressor(random_state=42)
elif best_model_name == "RandomForest":
    final_model = RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    )
elif best_model_name == "Linear":
    final_model = LinearRegression()

final_model.fit(X_full_scaled, y_full)

model_artifacts = {
    'model': final_model,
    'scaler': scaler,
    'features': best_features,
    'model_name': best_model_name,
    'threshold': best_threshold,
    'target': TARGET_COL
}

filename = f'best_model_{year}.joblib'
joblib.dump(model_artifacts, filename)
print(f"Model saved to: {filename}")


Best model: GradientBoost with 70% features
RMSE: 6.9673, R2: 0.3047
Model saved to: best_model_avg.joblib
